# What has the highest population in Washington out of battery EVs or plug-in EVs?


In [ ]:
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import scipy.stats as st
from config import gkey
import gmaps
import requests
import time

In [2]:
# Load the Size History by County csv to a DataFrame
size_history_data_path = "../cleaned_data/Electric_Vehicle_Population_Size_History_By_County_Cleaned.csv"
size_history_full_data = pd.read_csv(size_history_data_path)

In [3]:
size_history_full_data.head()

,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
0,January 31 2017,Juneau,AK,Passenger,0,1,1,49,50,2.00
1,January 31 2017,Dale,AL,Passenger,0,1,1,71,72,1.39
2,January 31 2017,Russell,AL,Passenger,1,0,1,53,54,1.85
3,January 31 2017,Maricopa,AZ,Passenger,2,0,2,1774,1776,0.11
4,January 31 2017,Pima,AZ,Passenger,0,1,1,439,440,0.23


In [4]:
# Refine to Washington st recent date in history (October 31 2021)
size_history_data_WA = size_history_full_data.loc[size_history_full_data["State"]=="WA"]
size_history_data_WA

,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
99,January 31 2017,Adams,WA,Passenger,2,1,3,12457,12460,0.02
100,January 31 2017,Adams,WA,Truck,0,0,0,5290,5290,0.00
101,January 31 2017,Asotin,WA,Passenger,4,4,8,12419,12427,0.06
102,January 31 2017,Asotin,WA,Truck,0,0,0,6648,6648,0.00
103,January 31 2017,Benton,WA,Passenger,129,133,262,129540,129802,0.20
...,...,...,...,...,...,...,...,...,...,...
11608,October 31 2021,Whatcom,WA,Truck,23,0,23,42447,42470,0.05
11609,October 31 2021,Whitman,WA,Passenger,69,57,126,21414,21540,0.58
11610,October 31 2021,Whitman,WA,Truck,2,0,2,8338,8340,0.02
11611,October 31 2021,Yakima,WA,Passenger,282,166,448,163726,164174,0.27


In [ ]:
# Washington State: Group dataframe into Counties, then summarise columns of interest
WA_county_data_EV_count = size_history_data.groupby("County")["Electric Vehicle (EV) Total"].sum()
county_data_total_count = size_history_data.groupby("County")["Total Vehicles"].sum()
county_data_percent = (county_data_EV_count / county_data_total_count)*100
county_data_percent = county_data_percent.map("{:.2f}".format)

# Assemble columns into dataframe
county_data = pd.DataFrame(
    {"Total Electric Vehicle (EV)": county_data_EV_count,
     "Total Vehicles": county_data_total_count,
     "Percent Electric Vehicles": county_data_percent
    }
)

# Sort data by descending order
county_data_sorted_percent = county_data.sort_values("Percent Electric Vehicles", ascending=False)
county_data_sorted_count = county_data.sort_values("Total Electric Vehicle (EV)", ascending=False)

# Display data
county_data_sorted_count

# Group dataframe into States, then summarise columns of interest
state_BEV_count = size_history_data.groupby("State")["Plug-In Hybrid Electric Vehicles (PHEVs)"].sum()
state_PHEV_count = size_history_data.groupby("State")["Battery Electric Vehicles (BEVs)"].sum()

# Assemble columns into dataframe
BEV_PHEV_data = pd.DataFrame(
    {"Total Plug-In Hybrid Electric Vehicles (PHEVs)": state_BEV_count,
     "Total Battery Electric Vehicles (BEVs)": state_PHEV_count
    }
)

# Display data
BEV_PHEV_data